In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
#Load the dataset:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458266 sha256=6e2f9cca36ed832000660a1364413657a1bae0125a2582c7882f28f6eaf286fd
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [5]:
# Initialize MPI
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [6]:
#Define the training function:
def train(model, x_train, y_train, rank, size):
    # Split the data across the nodes
    n =  len(x_train)
    chunk_size = n // size
    start = rank *  chunk_size
    end = (rank + 1) * chunk_size
    if rank == size - 1:
        end = n
    x_train_chunk = x_train[start:end]
    y_train_chunk = y_train[start:end]
    # Compile the model
    model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
    # Train the model
    model.fit(x_train_chunk, y_train_chunk, epochs=1, batch_size=32)
    # Compute the accuracy on the training data
    train_loss, train_acc = model.evaluate(x_train_chunk, y_train_chunk, verbose=2)
    # Reduce the accuracy across all nodes
    train_acc = comm.allreduce(train_acc, op=MPI.SUM)
    return train_acc / size

In [7]:
# Run the training loop:
epochs = 5
for epoch in range(epochs):
    # Train the model
    train_acc = train(model, x_train, y_train, rank, size)
    # Compute the accuracy on the test data
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    # Reduce the accuracy across all nodes
    test_acc = comm.allreduce(test_acc, op=MPI.SUM)
    # Print the results
    if rank ==  0:
        print(f"Epoch {epoch + 1}: Train accuracy = {train_acc:.4f}, Test accuracy = {test_acc / size:.4f}")

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.8802 - loss: 0.4142
1875/1875 - 9s - 5ms/step - accuracy: 0.9721 - loss: 0.0979
313/313 - 1s - 5ms/step - accuracy: 0.9705 - loss: 0.0969
Epoch 1: Train accuracy = 0.9721, Test accuracy = 0.9705
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 26s 13ms/step - accuracy: 0.9716 - loss: 0.0981
1875/1875 - 8s - 4ms/step - accuracy: 0.9806 - loss: 0.0656
313/313 - 2s - 7ms/step - accuracy: 0.9746 - loss: 0.0767
Epoch 2: Train accuracy = 0.9806, Test accuracy = 0.9746
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9820 - loss: 0.0637
1875/1875 - 9s - 5ms/step - accuracy: 0.9840 - loss: 0.0534
313/313 - 1s - 5ms/step - accuracy: 0.9780 - loss: 0.0692
Epoch 3: Train accuracy = 0.9840, Test accuracy = 0.9780
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9849 - loss: 0.0507
1875/1875 - 9s - 5ms/step - accuracy: 0.9884 - loss: 0.0399
313/313 - 2s - 5ms/step - accuracy: 0.9829 - loss: 0.0543
Epoch 4: Train accuracy = 0.9884, Test ac